# Elementary matrices, Gaussian elimination, reduced echelon form and row equivalence



In [1]:
include("linalg/rowequiv.jl")

using .RowEquivalence

## Identity matrix

The identity matrix is a square matrix in which all elements on the diagonal are 1 and all other elements in the matrix are 0.

Let's generate a $n \times n$ identity matrix by using `identitymatrix(n)`:

In [2]:
identitymatrix(3)

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

By default, the generated matrix will be a matrix over real numbers, implemented as floating-point numbers type `Float`. We can pass an additional parameter for the type, i.e. the algebraic structure over which we define the matrix. Here, we define a $2 \times 2$ identity matrix over rational numbers:

In [3]:
identitymatrix(2, Rational)

2×2 Matrix{Rational{Int64}}:
 1//1  0//1
 0//1  1//1

## Row / column operations and elementary matrices

Row operations perform one of the following three transformations to matrix defined over $F$:

- row switching: rows $i$ and $j$ are swapped
- row multiplication: row $i$ is multiplied by a scalar $s \in F$
- row addition: row $j$ multiplied by a scalar $s \in F$ and added to row $i$

Similar operations can be performed for columns.

Elementary matrices differ from the identity matrix by a single row operation. When multiplying an elementary matrix $E$ from left to a matrix $A$, this row operation is applied to $A$. When multiplying $E$ from right to a matrix $A$, this operation is applied as column operation to $A$.

Let's define a matrix `A` over integer numbers and apply row and column operations to it by multiplying elementary matrices with it:

In [4]:
A = [1  3  1  9;
     1  1 -1  1;
     3 11  5 35]

3×4 Matrix{Int64}:
 1   3   1   9
 1   1  -1   1
 3  11   5  35

We can generate an elementary matrix for row switching using `swapop_matrix(n, i, j, t)`, where `n` is the size (remember, the matrix is a square matrix), `i` and `j` are the row indices and `t` is the type.

Let's swap row 1 with row 3. `A` is an integer matrix and has 3 rows, so we must generate a $3 \times 3$ integer matrix:

In [5]:
E = swapop_matrix(3, 1, 3, Int64)

3×3 Matrix{Int64}:
 0  0  1
 0  1  0
 1  0  0

Left multiplication performs a row operation:

In [6]:
E * A

3×4 Matrix{Int64}:
 3  11   5  35
 1   1  -1   1
 1   3   1   9

When we want to swap columns, we need to generate a $4 \times 4$ matrix, since `A` has 4 columns:

In [7]:
E = swapop_matrix(4, 2, 4, Int64)

4×4 Matrix{Int64}:
 1  0  0  0
 0  0  0  1
 0  0  1  0
 0  1  0  0

Right multiplication now swaps columns 2 and 4:

In [8]:
A * E

3×4 Matrix{Int64}:
 1   9   1   3
 1   1  -1   1
 3  35   5  11

Row/column multiplication and addition can be done similarily:

In [10]:
# multiply row 2 by -1:
multop_matrix(3, 2, -1, Int64) * A

3×4 Matrix{Int64}:
  1   3  1   9
 -1  -1  1  -1
  3  11  5  35

In [11]:
# multiply row 1 by -1 and add this to row 2 (i.e. subtract row 1 from row 2)
addop_matrix(3, 2, 1, -1, Int64) * A

3×4 Matrix{Int64}:
 1   3   1   9
 0  -2  -2  -8
 3  11   5  35

## Gaussian elimination

[Gaussian elimination or more precisely Gauss-Jordan elimination](https://en.wikipedia.org/wiki/Gaussian_elimination) is used to tranform a matrix $A$ to its [reduced row echelon form](https://en.wikipedia.org/wiki/Row_echelon_form#Reduced_row_echelon_form) abbreviated as *REF* in the rest of this document. This transformation is done in a stepwise manner by performing elementary row operations, i.e. multiplying $A$ with elementary matrices as explained in the previous section.

In most cases, it is necessary that matrix is defined over a field, i.e. each element in the matrix besides zero must be invertible. This is for example the case for real numbers, but not for integer numbers, since the only invertible integers are -1 and 1 (for all others, you need fractions for inversion, which doesn't make them integers any more, e.g. $3^{-1} = \frac{1}{3}$).

The reason for this requirement is that in the REF the leading coefficients or *pivot* elements must be 1 and in order to achieve that, a row may be multiplied by the inverse of the leading coefficient of the input matrix.

This is also the reason why applying Gaussian elimination to an integer matrix may fail with `InexactError` in Julia:

In [12]:
reducedechelon_steps(A)

LoadError: InexactError: Int64(-0.5)

So let's convert $A$ to a matrix of real numbers:

In [14]:
mat = convert(Matrix{Float64}, A)

3×4 Matrix{Float64}:
 1.0   3.0   1.0   9.0
 1.0   1.0  -1.0   1.0
 3.0  11.0   5.0  35.0

The function `reducedechelon_steps()` can be used to get the resulting matrix at each step of the Gaussian elimination process. We also fetch the elementary matrices that were applied at each step via `return_operations=true`:

In [15]:
steps, ops = reducedechelon_steps(mat, return_operations=true)
length(ops)

5

We see that five steps were necessary to transform $A$ into its REF.

I defined a pretty printing helper function for matrices:

In [16]:
function prettyprint(x)
    show(stdout, "text/plain", x)
    println()
end

prettyprint (generic function with 1 method)

Now let's show the steps that were applied to $A$ and the resulting intermediate matrix at each step:

In [17]:
prevstep = mat
for (i, (step, op)) in enumerate(zip(steps, ops))
    println("-------")
    println("step ", i, ":")
    println("-------")
    println("apply")
    prettyprint(op)
    println("to")
    prettyprint(prevstep)
    println("=>")
    prettyprint(step)
    println()
    prevstep = step
end;

-------
step 1:
-------
apply
3×3 Matrix{Float64}:
  1.0  0.0  0.0
 -1.0  1.0  0.0
  0.0  0.0  1.0
to
3×4 Matrix{Float64}:
 1.0   3.0   1.0   9.0
 1.0   1.0  -1.0   1.0
 3.0  11.0   5.0  35.0
=>
3×4 Matrix{Float64}:
 1.0   3.0   1.0   9.0
 0.0  -2.0  -2.0  -8.0
 3.0  11.0   5.0  35.0

-------
step 2:
-------
apply
3×3 Matrix{Float64}:
  1.0  0.0  0.0
  0.0  1.0  0.0
 -3.0  0.0  1.0
to
3×4 Matrix{Float64}:
 1.0   3.0   1.0   9.0
 0.0  -2.0  -2.0  -8.0
 3.0  11.0   5.0  35.0
=>
3×4 Matrix{Float64}:
 1.0   3.0   1.0   9.0
 0.0  -2.0  -2.0  -8.0
 0.0   2.0   2.0   8.0

-------
step 3:
-------
apply
3×3 Matrix{Float64}:
  1.0   0.0   0.0
 -0.0  -0.5  -0.0
  0.0   0.0   1.0
to
3×4 Matrix{Float64}:
 1.0   3.0   1.0   9.0
 0.0  -2.0  -2.0  -8.0
 0.0   2.0   2.0   8.0
=>
3×4 Matrix{Float64}:
 1.0  3.0  1.0  9.0
 0.0  1.0  1.0  4.0
 0.0  2.0  2.0  8.0

-------
step 4:
-------
apply
3×3 Matrix{Float64}:
 1.0  -3.0  0.0
 0.0   1.0  0.0
 0.0   0.0  1.0
to
3×4 Matrix{Float64}:
 1.0  3.0  1.0  9.0
 0

We can see for example, that the first two steps subtracted the first row from the second and third row in order to achieve zeros below the first pivot element. Step three multiplied the second row with $-0.5$ to get the second pivot element, etc.

## Reduced echelon form

TODO

In [16]:
convert(Matrix{Int}, steps[end])

3×4 Matrix{Int64}:
 1  0  -2  -3
 0  1   1   4
 0  0   0   0

In [17]:
reducedechelon(mat)

3×4 Matrix{Float64}:
 1.0  0.0  -2.0  -3.0
 0.0  1.0   1.0   4.0
 0.0  0.0   0.0   0.0

## Row equivalence

TODO

## Matrix rank

TODO

## Matrix inverse

TODO

note that there are more efficient algorithms